In [1]:
import cv2
import cv2.aruco as aruco
import numpy as np

In [2]:
# Set the size of the calibration pattern (in meters)
patternSize = (0.02, 0.02)

# Set the number of inner corners in the calibration pattern
patternCorners = (9, 6)

In [3]:
# Create an array to store the 3D coordinates of the pattern points
patternPoints = np.zeros((np.prod(patternCorners), 3), np.float32)
patternPoints[:, :2] = np.indices(patternCorners).T.reshape(-1, 2)
patternPoints[:, :2] *= patternSize

# Create arrays to store the image points and object points
imagePoints = []
objectPoints = []

In [4]:
# Open a video capture object
cap = cv2.VideoCapture(0)

In [5]:
# Set a flag to indicate whether calibration is done
calibrationDone = False

while not calibrationDone:
    # Capture a frame from the camera
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners in the frame
    ret, corners = cv2.findChessboardCorners(gray, patternCorners)

    # If the chessboard corners were found
    if ret:


        # Refine the corner locations
        corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1))

        # Draw the chessboard corners on the frame
        cv2.drawChessboardCorners(frame, patternCorners, corners, ret)

        # Add the image points and object points to their respective arrays
        imagePoints.append(corners)
        objectPoints.append(patternPoints)

        # Check if enough points have been collected for calibration
        if len(imagePoints) >= 10:
            # Calibrate the camera using the collected points
            ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objectPoints, imagePoints, gray.shape[::-1], None, None)

            # Extract the focal length from the camera matrix
            focalLength = mtx[0][0]

            # Save the camera matrix and distortion coefficients to a file
            np.savez("calibration.npz", mtx=mtx, dist=dist)
            
            
            # Set the flag to indicate that calibration is done
            calibrationDone = True





In [6]:
# Specify the ArUco dictionary
arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_100)

# Define the ArUco detection parameters
arucoParams = cv2.aruco.DetectorParameters_create()

# Set the size of the marker (in meters)
markerSize = 0.10


while True:
    ret, frame = cap.read()

    # Detect the ArUco markers in the frame
    corners, ids, _ = cv2.aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)

    if ids is not None:
        # Estimate the pose of each detected marker
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, markerSize, mtx, dist)

        # Loop over each detected marker
        for i in range(len(ids)):
            # Calculate the distance of the marker from the camera
            distance = np.linalg.norm(tvecs[i])

            # Calculate the confidence level based on the distance
            confidence = 10  # Assuming maximum confidence since we don't have reprojection error

            # Get the corner coordinates of the marker
            corner = corners[i][0]

            # Draw a label with the distance and confidence level on top of the marker
            cv2.putText(frame, f"{distance:.2f}m (confidence: {confidence})",
                        (int(corner[0]), int(corner[1]) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Draw the detected markers on the frame
    frame = cv2.aruco.drawDetectedMarkers(frame, corners, ids)

    # Show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # If the 'q' key is pressed or if 'ESC' key is pressed, stop the loop.
    if key == ord("q") or key == 27:
        break


TypeError: only size-1 arrays can be converted to Python scalars

In [7]:
# Release the video capture object
cap.release()

# Close all windows
cv2.destroyAllWindows()